# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [162]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
import statsmodels.formula.api as smf 


In [163]:
df_raw = pd.read_csv("admissions.csv")
df = df_raw.dropna() 
df.head()

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [164]:
# frequency table for prestige and whether or not someone was admitted

pd.crosstab(df.admit, columns ='prestige')

col_0,prestige
admit,
0,271
1,126


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [165]:
dummy_r = pd.get_dummies(df['prestige'], prefix='prestige')
dummy_r.head()

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: In this case 3 (4-1)

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [166]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_r.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           0.0           1.0           0.0
1      1  660.0  3.67           0.0           0.0           1.0           0.0
2      1  800.0  4.00           1.0           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           0.0           1.0


In [167]:
#ODDS RATIO

lreg8 = smf.logit(formula = 'admit ~ gre + gpa + C(prestige)', data = df).fit()

# odds ratios
print ("Odds Ratios")
print (np.exp(lreg8.params))

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6
Odds Ratios
Intercept             0.020716
C(prestige)[T.2.0]    0.506548
C(prestige)[T.3.0]    0.262192
C(prestige)[T.4.0]    0.211525
gre                   1.002221
gpa                   2.180027
dtype: float64


In [168]:
# crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

df4 = pd.crosstab(df['admit'], df['prestige'], rownames=['admit'])
print df4

prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [169]:
#ODDS RATIO
#lreg9 = smf.logit(formula = 'admit ~ gre + gpa + C(prestige, Treatment(reference=2))', data = df).fit()
#odds ratios
#print ("Odds Ratios")
#print (np.exp(lreg9.params))

#GLM
logit2 = sm.GLM(df2['admit'], df2[df2.columns[3]], family=sm.families.Binomial())
#ODDS RATIOS
print "Odds Ratio:"
print np.exp(logit2.fit().params)

Odds Ratio:
prestige_1.0    1.178571
dtype: float64


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [170]:
#removing one dummy variable
df3 = df[['admit', 'gre', 'gpa']].join(dummy_r.ix[:, 'prestige_2':])
df3.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0.0,1.0,0.0
1,1,660.0,3.67,0.0,1.0,0.0
2,1,800.0,4.00,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,1.0


In [171]:
#GLM
logit3 = sm.GLM(df3['admit'], df3[df3.columns[3:]], family=sm.families.Binomial())

In [172]:
#ODDS RATIO
print "Odds Ratio:"
print np.exp(logit3.fit().params)

Odds Ratio:
prestige_2.0    0.557895
prestige_3.0    0.301075
prestige_4.0    0.218182
dtype: float64


#### 3.3 Calculate the odds ratio

In [173]:
lreg8 = smf.logit(formula = 'admit ~ gre + gpa + C(prestige)', data = df).fit()
# odds ratios
print ("Odds Ratios")
print (np.exp(lreg8.params))

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6
Odds Ratios
Intercept             0.020716
C(prestige)[T.2.0]    0.506548
C(prestige)[T.3.0]    0.262192
C(prestige)[T.4.0]    0.211525
gre                   1.002221
gpa                   2.180027
dtype: float64


#### 3.4 Write this finding in a sentenance: 

Answer: the association between "admit" and "prestige_2, prestige_3 or prestige_4" is ("<0") lower than "admit" vs "prestige_1" (">1)

#### 3.5 Print the cross tab for prestige_4

In [174]:
dfnew = pd.DataFrame(data=df, columns=['admit', 'gpa', 'gre', 'prestige'])
just_dummies = pd.get_dummies(df['prestige'])

df10 = pd.concat([df, just_dummies], axis=1)      
df10.drop(['prestige', 1] , inplace=True, axis=1)


df11 = df10.rename(columns={ 2.0 : 'p2', 3.0 : 'p3', 4.0 : 'p4'})


df11.head()

,admit,gre,gpa,p2,p3,p4
0,0,380.0,3.61,0.0,1.0,0.0
1,1,660.0,3.67,0.0,1.0,0.0
2,1,800.0,4.00,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,1.0


In [175]:
df13 = pd.crosstab(df['admit'], df['p2'], rownames=['admit'])
print df13

## Here I do not know/understand the reason of this error...

KeyError: 'p2'

#### 3.6 Calculate the OR 

In [ ]:
NA

#### 3.7 Write this finding in a sentence

Answer: NA

## Part 4. Analysis

In [176]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_r.ix[:, 'prestige_2':]) #removing one dummy variable
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           1.0           0.0
1      1  660.0  3.67           0.0           1.0           0.0
2      1  800.0  4.00           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [177]:
# manually add the intercept
data['intercept'] = 1.0
data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,0,380.0,3.61,0.0,1.0,0.0,1.0
1,1,660.0,3.67,0.0,1.0,0.0,1.0
2,1,800.0,4.00,0.0,0.0,0.0,1.0
3,1,640.0,3.19,0.0,0.0,1.0,1.0
4,0,520.0,2.93,0.0,0.0,1.0,1.0


#### 4.1 Set the covariates to a variable called train_cols

In [178]:
train_cols = data.columns[1:]

#### 4.2 Fit the model

In [179]:
logit4 = sm.Logit(data['admit'], data[train_cols])

#### 4.3 Print the summary results

In [180]:
result = logit4.fit()
print result.summary()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Thu, 21 Jul 2016   Pseudo R-squ.:                 0.08166
Time:                        02:59:49   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [181]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: the chances of being admitted decreases by about 50% if the prestige is 2

#### 4.6 Interpret the OR of GPA

Answer: if the gpa increases students might be 2.1 times more likely to be admited

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [182]:

def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [183]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [184]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']


# keep only what we need for making predictions

cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy.ix[:, 'prestige_2':])

#### 5.2 Make predictions on the enumerated dataset

In [185]:
combos['admit_pred'] = result.predict(combos[train_cols])

KeyError: "Index([u'prestige_2.0', u'prestige_3.0', u'prestige_4.0'], dtype='object') not in index"

#### 5.3 Interpret findings for the last 4 observations

Answer: same problem as exercise 3.5

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.